In [ ]:
import os
import cv2

import numpy as np
import pandas as pd
from PIL import Image
from numpy import random

from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from PIL import ImageDraw, ImageFont, Image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import model_from_json
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, Flatten

## Read in data

In [ ]:
## Read in the seismic image which has been converted into ASCII format

input_path = '...'

Data=np.genfromtxt(input_path + ".../97_09_01_curv3d_mostpos_il_data.ascii",delimiter ='')
Labeld=np.genfromtxt(input_path + ".../97_09_01_curv3d_mostpos_il_fault.ascii",delimiter ='')

## Data Preparation

In [ ]:
## Extracting Subline, Crossline, Time, Amplitude (data in ascii)

Subline = Data[1:,0:1]
Crossline = Data[1:,1:2]
Time = Data[1:,2:3]
Amp = Data[1:,3:4]
Label = Labeld[1:,3:4]

print(len(Subline),len(Crossline),len(Time),len(Amp))
print(len(Label))

In [ ]:
## Extracting only one subline

xline=4000
times=351
subline=9
A=Amp[(subline-1)*xline*times:subline*xline*times,:]

print(A.shape)

t=1
a=np.ones((times,1))
print(a.shape)
for i in range(xline):
    b=0
    temp=A[i*times:t*times,0*t:1*t]
    b=t*20
    t=t+1
    a=np.c_[a,temp]

a=a[:,1:4001]


print(a.shape)

A=a*255
A=Image.fromarray(A)
A.show()


print(a.shape)



L=Label[(subline-1)*xline*times:subline*xline*times,:]

print(L.shape)

t=1
a=np.ones((times,1))
print(a.shape)
for i in range(xline):
    b=0
    temp=L[i*times:t*times,0*t:1*t]
    b=t*20
    t=t+1
    a=np.c_[a,temp]

a=a[:,1:4001]
print(a.shape)

L=a*255
L=Image.fromarray(L)
L.show()

In [ ]:
## Sliding window to crop to arrange the dataset in array form

L1=L.crop((2170,50,3311,275))
L1.show()
A1=A.crop((2170,50,3311,275))
A1.show()

subline1=np.asarray(A1)
h,w=subline1.shape
Label1=np.asarray(L1)

# Fault=np.ones((40,40))
# No_Fault=np.ones((40,40))
# img_id=0

In [ ]:
## Export the crop image into folders in computer

t=10
t2=3

h1=0
h2=40
run2=True
while run2==True:
    w1=0
    w2=40
    print(h1,h2,w1,w2)
    run1=True
    while run1==True:
        label_sw=Label1[h1:h2,w1:w2]
        data=subline1[h1:h2,w1:w2]
        img=Image.fromarray(data)
        img=img.convert('L')
        if np.sum(label_sw)>=t:
            img_id+=1
            Fault=np.dstack((Fault,data))
            img.save('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/fault/'+str(img_id)+'.png')
        elif np.sum(label_sw)<t2:
            img_id+=1
            No_Fault=np.dstack((No_Fault,data))
            img.save('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/no_fault/'+str(img_id)+'.png')
        
        w1+=10
        w2+=10
        if w2>w:
            data=subline1[h1:h2,w-40:w]
            img=Image.fromarray(data)
            img=img.convert('L')
            if np.sum(label_sw)>=t:
                Fault=np.dstack((Fault,data))
                img_id+=1
                img.save('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/fault/'+str(img_id)+'.png')
                break
            elif np.sum(label_sw)<t2:
                img.save('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/no_fault/'+str(img_id)+'.png')
                No_Fault=np.dstack((No_Fault,data))
                img_id+=1
                break
    h1+=10
    h2+=10
    if h2>h :
        if w2<w:
            data=subline1[h-40:h,w1:w2]
        else:
            data=subline1[h-40:h,w-40:w]
        img=Image.fromarray(data)
        img=img.convert('L')
        if np.sum(label_sw)>=t:
            img.save('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/fault/'+str(img_id)+'.png')
            Fault=np.dstack((Fault,data))
            img_id+=1
            break
        elif np.sum(label_sw)<t2:
            img.save('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/no_fault/'+str(img_id)+'.png')
            No_Fault=np.dstack((No_Fault,data))
            img_id+=1
            break

#### Transpose the data to generate more training dataset

In [140]:
print(Fault.shape)
Fault_T=np.transpose(Fault,(2,0,1))
No_Fault_T=np.transpose(No_Fault,(2,0,1))
print(Fault_T.shape)
print(No_Fault_T.shape)
# print(Fault_T)

(40, 40, 2703)
(2703, 40, 40)
(19554, 40, 40)


In [141]:
Fault_data=Fault_T[1:,:]
No_Fault_data=No_Fault_T[1:,:]
Fault_data.tofile('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/Total_Fault__train1.txt',sep=",",format="%s")
No_Fault_data.tofile('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/Total_NoFault_train1.txt',sep=",",format="%s")
# # np.savetxt('Total_Fault.txt',No_Fault)
# # np.savetxt('No_Fault.txt',No_Fault)
# print(Fault.shape)


In [142]:
# # Fault_train=np.genfromtxt("/Users/rewathi/Desktop/Hackathon/Img/subline_seis/Total_Fault_1.txt",delimiter =',')
# No_Fault_train=np.genfromtxt("/Users/rewathi/Desktop/Hackathon/Img/subline_seis/Total_NoFault_1.txt",delimiter =',')
# Fault=Fault_train.reshape((1787,40,40))
print(Fault_data.shape)

(2702, 40, 40)


In [143]:
#making training dataset
TrainF=Fault_data[:2702,:]
TrainNF=No_Fault_data[:2702,:]
print(TrainNF.shape)
Train=np.r_[(TrainF,TrainNF)]
Train.tofile('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/Train1.txt',sep=",",format="%s")
print(Train.shape)
Labeltrain1=np.ones((2702))
Labeltrain2=np.zeros((2702))
Labeltrain=np.r_[(Labeltrain1,Labeltrain2)]
np.savetxt('/Users/rewathi/Desktop/CHECK_data/sublines_ascii/Train1_Label.txt',Labeltrain,delimiter=',')

(2702, 40, 40)
(5404, 40, 40)


In [276]:
TestF=Fault[855:1055,:]
TestNF=No_Fault[855:1055,:]
Test=np.r_[(TestF,TestNF)]
print(Test.shape)
Test.tofile('Test.txt',sep=",",format="%s")

(400, 20, 20)


In [296]:
Labeltest1=np.ones((200))
Labeltest2=np.zeros((200))
Labeltest=np.r_[(Labeltest1,Labeltest2)]
np.savetxt('Test_Label.txt',Labeltest,delimiter=',')

## Modelling

In [ ]:
## Read into train and test dataset for modelling
input_path = '...'

## Train data
x_train = np.genfromtxt(input_path +  ".../XL_train1.txt", delimiter = ',')
y_train = np.genfromtxt(input_path +  ".../XL_train1_Label.txt", delimiter = ',')

## Test data
x_test = np.genfromtxt(input_path +  ".../Test_train1.txt", delimiter = ',')
y_test = np.genfromtxt(input_path +  ".../Test_train1_Label.txt", delimiter = ',')

In [ ]:
## Reshape the data for input into modelling

x_train = (x_train/255).reshape(4712, 40, 40)
y_train = y_train
x_test = (x_test/255).reshape(714, 40, 40)
y_test = y_test

In [ ]:
## Baseline modelling

#network structure
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(40, 40)),
    tf.keras.layers.Dense(200, activation=tf.nn.relu),
    tf.keras.layers.Dense(400, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
## Modelling
baseline_model = model.fit(x_train_3d, y_train_3d, epochs=500, validation_data=(x_test_3d, y_test_3d))

In [ ]:
## Visualize the model accuracy and loss function

fig = plt.figure(figsize = (20,8))
plt.subplot(2,1,1)
plt.plot(baseline_model.history['acc'])
plt.plot(baseline_model.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(baseline_model.history['loss'])
plt.plot(baseline_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.tight_layout()